In [40]:
import requests
from bs4 import BeautifulSoup
import paramiko
import time

In [41]:
url = 'http://pre-spark-02-swi.hitt.hre.local:8080/'
response = requests.get(url)
response

<Response [200]>

In [42]:
streaming_page_data = BeautifulSoup(response.text, 'html.parser')
table = streaming_page_data.find_all('table',  class_ =  "table table-bordered table-condensed table-striped sortable")
table

[<table class="table table-bordered table-condensed table-striped sortable">
 <thead><th class="" width="">Worker Id</th><th class="" width="">Address</th><th class="" width="">State</th><th class="" width="">Cores</th><th class="" width="">Memory</th></thead>
 <tbody>
 <tr>
 <td>
 <a href="http://172.31.52.36:8082">
               worker-20240403122225-172.31.52.36-40291
             </a>
 </td>
 <td>172.31.52.36:40291</td>
 <td>ALIVE</td>
 <td>35 (5 Used)</td>
 <td sorttable_customkey="43949.7168">
         42.9 GB
         (7.0 GB Used)
       </td>
 </tr><tr>
 <td>
 <a href="http://172.31.52.37:8082">
               worker-20240403122236-172.31.52.37-33231
             </a>
 </td>
 <td>172.31.52.37:33231</td>
 <td>ALIVE</td>
 <td>35 (3 Used)</td>
 <td sorttable_customkey="43949.3072">
         42.9 GB
         (3.0 GB Used)
       </td>
 </tr><tr>
 <td>
 <a href="http://172.31.52.38:8082">
               worker-20240403122244-172.31.52.38-44719
             </a>
 </td>
 <td>172.31.

In [53]:
data = []
hostname = '172.30.52.36'
port = 22
username = 'pre-ansible-core'
password = '+Ch3esePle4s3!'  
streaming_app = ['Virdict Data Processing - IEP-GW' , 'Virdict Data Processing - SE' , 'Virdict Data Processing - IEP-OTHER' , 'Virdict Data Processing - ASR' , 'Virdict Data Processing - IEP-LNER']
for row in table[1].find_all('tr'):
    cells = row.find_all('td')
    if cells: 
        data_row = [cell.text.strip() for cell in cells]
        data.append(data_row)
if len(data) < 5:
    for it in streaming_app:
        if len([i for i in data if i[1] == it]) == 0:
            print(it)
            ssh = paramiko.SSHClient()
            ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            try:
                ssh.connect(hostname, port, username, password)
                command = f"sudo systemctl restart virdictstreaming_{it[26:]}.service"
                print(command)
                channel = ssh.invoke_shell()
                channel.send(command + '\n')
                time.sleep(2)
                channel.send(password + '\n')
                time.sleep(2)
            finally:
                # Đóng kết nối
                ssh.close()

Virdict Data Processing - IEP-GW
sudo systemctl restart virdictstreaming_IEP-GW.service
Virdict Data Processing - IEP-LNER
sudo systemctl restart virdictstreaming_IEP-LNER.service


In [50]:
streaming_app[0][26:]

'IEP-GW'